### 0.Import

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

### 1. combine mon_features.pkl & unmon_features.pkl into features_df

In [2]:
import pandas as pd
import numpy as np
import pickle

MON_FILE_PATH = '/content/mon_features.pkl'
UNMON_FILE_PATH = '/content/unmon_features.pkl'

LABEL_COLUMN = ['website_label', 'monitored_label']

FEATURES_VER3 = [
    'total_transmission_time', 'std_inter_packet_time',
    'avg_outgoing_burst_size', 'avg_incoming_burst_size',
    'num_outgoing_packets', 'incoming_packet_ratio',
    'outgoing_packet_ratio', 'cumul_packets_10pct',
    'cumul_packets_30pct', 'outgoing_order_skew',
    'incoming_order_skew', 'cumul_max', 'bigram_OO',
    'num_incoming_first_30', 'outgoing_first_30',
    'avg_incoming_order_first_30', 'avg_outgoing_order_first_30'
]

mon_features_df = pd.read_pickle(MON_FILE_PATH)
unmon_features_df = pd.read_pickle(UNMON_FILE_PATH)

features_df = pd.concat([mon_features_df, unmon_features_df], ignore_index=True)

X = features_df[FEATURES_VER3]
y = features_df[LABEL_COLUMN[0]]


# replace unmonitored label(-1) into 95
y = y.replace({-1: 95})

print(X)
print(y)

       total_transmission_time  std_inter_packet_time  \
0                        10.14               0.041168   
1                        10.16               0.163930   
2                        11.11               0.066661   
3                        13.36               0.047809   
4                        10.64               0.038760   
...                        ...                    ...   
28995                    32.09               0.163669   
28996                    38.62               0.114350   
28997                    34.93               1.331199   
28998                    11.84               0.083521   
28999                     9.62               0.026874   

       avg_outgoing_burst_size  avg_incoming_burst_size  num_outgoing_packets  \
0                     1.551282                16.666667                 121.0   
1                     1.702128                 9.319149                  80.0   
2                     1.552632                16.315789                 

In [3]:
# 로그 변환
log_cols = []

for col in X.columns:
    # 숫자형이고 & 음수가 하나도 없는 경우만 로그 변환 리스트에 추가
    if pd.api.types.is_numeric_dtype(X[col]) and (X[col] < 0).sum() == 0:
        log_cols.append(col)

# 리스트에 있는 컬럼들 로그 변환
if log_cols:
    X[log_cols] = np.log1p(X[log_cols])

/tmp/ipython-input-1747048184.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[log_cols] = np.log1p(X[log_cols])


### 2. model [K-NN]

#### default 학습

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
# Scaling 추가
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [7]:
y_pred = knn.predict(X_test)
y_pred_proba = knn.predict_proba(X_test)

In [8]:
acc = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class="ovr")
report = classification_report(y_test, y_pred)

print("=== K-NN 기본 성능 ===")
print("Accuracy:", acc)
print("ROC-AUC:", roc_auc)
print("\nClassification Report:\n", report)

=== K-NN 기본 성능 ===
Accuracy: 0.7582758620689655
ROC-AUC: 0.9333496349597953

Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.53      0.50        40
           1       0.76      0.85      0.80        40
           2       0.62      0.72      0.67        40
           3       0.74      0.78      0.76        40
           4       0.57      0.72      0.64        40
           5       0.54      0.72      0.62        40
           6       0.71      0.90      0.79        40
           7       0.56      0.62      0.59        40
           8       0.52      0.62      0.57        40
           9       0.49      0.65      0.56        40
          10       0.66      0.72      0.69        40
          11       0.62      0.62      0.62        40
          12       0.83      0.88      0.85        40
          13       0.48      0.38      0.42        40
          14       0.46      0.65      0.54        40
          15       0.68      0.70 

#### 하이퍼 파라미터 튜닝 후 학습(GridSearchCV 이용)

In [9]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 15, 21],
    'weights': ['uniform', 'distance'],
    'metric': ['minkowski', 'manhattan', 'euclidean']
}

grid = GridSearchCV(knn, param_grid, cv=5, scoring='f1_macro')
grid.fit(X_train, y_train)

print(grid.best_params_)
print(grid.best_estimator_)

grid_pred = grid.predict(X_test)
grid_pred_proba = grid.predict_proba(X_test)

{'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
KNeighborsClassifier(metric='manhattan', n_neighbors=3, weights='distance')


In [10]:
acc = accuracy_score(y_test, grid_pred)
roc_auc = roc_auc_score(y_test, grid_pred_proba, multi_class="ovr")
report = classification_report(y_test, grid_pred)

print("=== K-NN 튜닝 후 성능 ===")
print("Accuracy:", acc)
print("ROC-AUC:", roc_auc)
print("\nClassification Report:\n", report)

=== K-NN 튜닝 후 성능 ===
Accuracy: 0.825
ROC-AUC: 0.9371266175054064

Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.65      0.68        40
           1       0.76      0.80      0.78        40
           2       0.80      0.88      0.83        40
           3       0.87      0.85      0.86        40
           4       0.72      0.70      0.71        40
           5       0.83      0.88      0.85        40
           6       0.80      0.90      0.85        40
           7       0.79      0.78      0.78        40
           8       0.69      0.68      0.68        40
           9       0.74      0.80      0.77        40
          10       0.78      0.78      0.78        40
          11       0.83      0.75      0.79        40
          12       0.90      0.90      0.90        40
          13       0.68      0.57      0.62        40
          14       0.74      0.70      0.72        40
          15       0.74      0.80      0.77  